## Hydroponic optimization strategies for simultanious nutrient and pH control

#### Hydroponic nutriet pollution results from frequent replacement of the nutrient solution as dictated by nutrient depletion and inert buildup. This can be minimized by controlling the nutrient concentrations at constant levels and avoiding the addition of inert species such as Na and Cl. Nutrient concentration control can be easily accomplished through the use of EC, but limiting the addition of inert species is tricky. This is partly due to nutriet addition constraints resulting from available salts. For example, if the nutrient dosing solution (used to control the nutrient concentrations) is lacking $\mathrm{Ca}$, either $\mathrm{CaNO_3}$ or $\mathrm{CaCl_2}$ must be added, since complexes with $\mathrm{Mg}$ and $\mathrm{PO_4}$ are insoluble. The dosing solution may then have too much $\mathrm{NO_3}$ or $\mathrm{Cl}$, causing accumulation. 


#### Furthermore, using standard Hoagland's solution (with nitrate as the sole nitrogen source), for example, causes a rise in pH, which requires acid dosing and hence the addition of another anion (like $\mathrm{Cl^-}$, $\mathrm{NO_3^-}$ or $\mathrm{SO_4^-}$). Therefore, a calculation is required to compose a dosing solution which will dose nutirents and protons (acid) at the same rate at which the plants consume them, without the buildup of inetrs species like Na or Cl. The total nutrient concnetration and pH can then be simultaneously controlled (using EC) with perhaps only small additions of $\mathrm{HCl}$ or $\mathrm{NaOH}$ to account for errors in the dosing solution makeup (if the pH drifts). Note that a nutrient+acid dosing solution with the exact proportions of nutrients and protons may not be possible due to limited degrees of freedom (limited soluble salts) and hence an optimization algorithm is required rather than a root finding algorithm. This allows for a best-possible dosing solution makeup to be calculated

#### Note that nutriet uptake is modeled with the co-absorption of protons (hence the rise in pH). Using the charge balance constraint, neglecting the micronutrients and assuming constant nutrient uptake ratios, the total number of protons absorbed can be estimated from the amounts of other nutreints absorbed:

#### $$\mathrm{H^+} = \mathrm{NO_3^-} + 2\mathrm{SO_4^{2-}} + \mathrm{H_2PO_4^-} - \mathrm{K^+} - 2\mathrm{Ca^{2+}} - 2\mathrm{Mg^{2+}}$$

#### Defining the variables (nutrient salts, acids and bases), in mM

###### $\mathrm{C1 ~~~=~ KNO_3}$                       
###### $\mathrm{C2 ~~~=~ K_2SO_4}$                         
###### $\mathrm{C3 ~~~=~ KH_2SO_4}$              
###### $\mathrm{C4 ~~~=~ Mg(NO_3)_2\bullet 6H_2O}$                
###### $\mathrm{C5 ~~~=~ MgSO_4\bullet 7H_2O}$                
###### $\mathrm{C6 ~~~=~ Ca(NO_3)\bullet 4H_2O}$                 
###### $\mathrm{C7 ~~~=~ 70~\%~HNO_3}$                    
###### $\mathrm{C8 ~~~=~ 98~\%~H_2SO_4}$         
###### $\mathrm{C9 ~~~=~ CaCl_2\bullet 2H_2O}$              
###### $\mathrm{C10  ~=~ NaNO_3}$            
###### $\mathrm{C11  ~=~ KCl}$
###### $\mathrm{C12  ~=~ 37~\%~HCl}$
###### $\mathrm{C13  ~=~ KOH}$
###### $\mathrm{C14  ~=~ Ca(OH)_2}$
###### $\mathrm{C15  ~=~ NaOH}$
###### $\mathrm{C16  ~=~ K_2HSO_4}$
###### $\mathrm{C17  ~=~ K_3PO_4}$
###### $\mathrm{C18  ~=~ 85~\%~H_3PO_4}$
###### $\mathrm{C19  ~=~ NaCl}$



#### Molar masses in g/mol (or in ml/mol where specified for liquids)

In [5]:
MM1  = 101.1032           
MM2  = 174.259               
MM3  = 136.086              
MM4  = 256.14             
MM5  = 246.48             
MM6  = 236.15             
MM7  = 63.291     # ml/mol    
MM8  = 54.348     # ml/mol    
MM9  = 147.01                
MM10 = 84.99              
MM11 = 74.5513            
MM12 = 1/0.012    # ml/mol  
MM13 = 56.1               
MM14 = 74.1               
MM15 = 40                 
MM16 = 174.2              
MM17 = 212.27             
MM18 = 1/0.0146   # ml/mol        
MM19 = 58.44              

#### Specifing the amounts of nutrients that should be in the dosing solution (proportional to the plant uptake rates), in mM

In [6]:
N  = 100         #  Nitrogen
K  = N*0.375     #  Potassium
Ca = N*0.2       #  Calcium
P  = N*0.075     #  Phosphate
S  = N*0.1       #  Sulphate
Mg = N*0.05      #  Magnesium
H  = N*0.5       #  Protons
Cl = N*0.05      #  Chlorine
Na = 0           #  Sodium

#### To help stabilize the optimization algorithm, zero overall charge can be ensured by adding additional $\mathrm{Na^+}$ or $\mathrm{Cl^-}$ (since the final solution will always be neutral):

In [7]:
charge = -N + K + 2*Ca - P - 2*S + 2*Mg + H - Cl

if charge > 0:      # add additional Cl
    Cl += charge
    Na = 0         
elif charge < 0:    # add additional Na
    Na = charge

#### An optimization algoritm that minimizes the objective function is selected. scipy.optimize.minimize. The objective function can then simply be the added nutrients minus the specified nutrients. The value of the objective function will then be zero if the correct amount of nutrients has been added (minima). For example: Considering nitrogen only, if the objective function equals C1 + C6 - N (see previous cells). Minimization will yield the most-correct amounts of C1 and C6 to be added to the dosing solution.

In [8]:
def ObjectiveFunction(chem):
    C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, C11, C12, C13, C14, C15, C16, C17, C18, C19 = chem    # Nutrinet salts/acids/bases 
    
    eq1 = abs(C1 + 2*C6 + 2*C4 + C10 + C7                               - N)                       #  N balance
    eq2 = abs(C1 + C3 + 2*C2 + C11 + C13 + 2*C16 + 3*C17                - K)                       #  K balance
    eq3 = abs(C6 + C9 + C14                                             - Ca)                      #  Ca balance
    eq4 = abs(C3 + C16 + C17 + C18                                      - P)                       #  P balance
    eq5 = abs(C5 + C2 + C8                                              - S)                       #  S balance
    eq6 = abs(C5 + C4                                                   - Mg)                      #  Mg balance
    eq7 = abs(C7 + C12 + 2*C8 - C13 - 2*C14 - C15 - C16 - 2*C17 + 2*C18 - H)                       #  H balance
    eq8 = abs(2*C9 + C11 + C12 + C19                                    - Cl)                      #  Cl balance
    eq9 = abs(C10 + C15 + C19                                           - Na)                      #  Na balance
    
    obj = eq1 + eq2 + eq3 + eq4 + eq5 + eq6 + eq7 + eq8 + eq9
    
    return obj

#### Using scipy.optimize.minimize, the following cell simply defines some required inequality constraints (preventing negative values):

In [9]:
def con1(chem):
    return chem[0]
def con2(chem):
    return chem[1]
def con3(chem):
    return chem[2]
def con4(chem):
    return chem[3]
def con5(chem):
    return chem[4]
def con6(chem):
    return chem[5]
def con7(chem):
    return chem[6]
def con8(chem):
    return chem[7]
def con9(chem):
    return chem[8]
def con10(chem):
    return chem[9]
def con11(chem):
    return chem[10]
def con12(chem):
    return chem[11]
def con13(chem):
    return chem[12]
def con14(chem):
    return chem[13]
def con15(chem):
    return chem[14]
def con16(chem):
    return chem[15]
def con17(chem):
    return chem[16]
def con18(chem):
    return chem[17]
def con19(chem):
    return chem[18]

cons1 =  {'type': 'ineq', 'fun': con1}
cons2 =  {'type': 'ineq', 'fun': con2}
cons3 =  {'type': 'ineq', 'fun': con3}
cons4 =  {'type': 'ineq', 'fun': con4}
cons5 =  {'type': 'ineq', 'fun': con5}
cons6 =  {'type': 'ineq', 'fun': con6}
cons7 =  {'type': 'ineq', 'fun': con7}
cons8 =  {'type': 'ineq', 'fun': con8}
cons9 =  {'type': 'ineq', 'fun': con9}
cons10 = {'type': 'ineq', 'fun': con10}
cons11 = {'type': 'ineq', 'fun': con11}
cons12 = {'type': 'ineq', 'fun': con12}
cons13 = {'type': 'ineq', 'fun': con13}
cons14 = {'type': 'ineq', 'fun': con14}
cons15 = {'type': 'ineq', 'fun': con15}
cons16 = {'type': 'ineq', 'fun': con16}
cons17 = {'type': 'ineq', 'fun': con17}
cons18 = {'type': 'ineq', 'fun': con18}
cons19 = {'type': 'ineq', 'fun': con19}

#### Searching for minima in the objective function:

In [10]:
from scipy.optimize import minimize

guess = [20, 0, 2, 0, 1, 10, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
consts = (cons1, cons2, cons3, cons4, cons5, cons6, cons7, cons8, cons9, cons10,\
          cons11, cons12, cons13, cons14, cons15, cons16, cons17, cons18, cons19)


f = minimize(ObjectiveFunction, guess, constraints=consts)


out = f['x']
for i in range(len(out)):
    if out[i] <= 0:
        out[i] = 0

print('convergence: ', f['success'])
print('objective function value = ', round(f['fun'], 6))
print()
print('The dosing solution should contain:')
print()
print('C1   KNO3   = {0}   g / L'.format(round(out[0] *MM1 /1000, 2)))
print('C2   K2SO4  = {0}   g / L'.format(round(out[1] *MM2 /1000, 2)))
print('C3   KH2PO4 = {0}   g / L'.format(round(out[2] *MM3 /1000, 2)))
print('C4   MgNO3  = {0}   g / L'.format(round(out[3] *MM4 /1000, 2)))
print('C5   MgSO4  = {0}   g / L'.format(round(out[4] *MM5 /1000, 2)))
print('C6   CaNO3  = {0}   g / L'.format(round(out[5] *MM6 /1000, 2)))
print('C7   HNO3   = {0}  ml / L'.format(round(out[6] *MM7 /1000, 2)))
print('C8   H2SO4  = {0}  ml / L'.format(round(out[7] *MM8 /1000, 2)))
print('C9   CaCl2  = {0}   g / L'.format(round(out[8] *MM9 /1000, 2)))
print('C10  NaNO3  = {0}   g / L'.format(round(out[9] *MM10/1000, 2)))
print('C11  KCl    = {0}   g / L'.format(round(out[10]*MM11/1000, 2)))
print('C12  HCl    = {0}  ml / L'.format(round(out[11]*MM12/1000, 2))) 
print('C13  KOH    = {0}   g / L'.format(round(out[12]*MM13/1000, 2)))
print('C14  CaOH   = {0}   g / L'.format(round(out[13]*MM14/1000, 2)))
print('C15  NaOH   = {0}   g / L'.format(round(out[14]*MM15/1000, 2)))
print('C16  K2HPO4 = {0}   g / L'.format(round(out[15]*MM16/1000, 2)))
print('C17  K3PO4  = {0}   g / L'.format(round(out[16]*MM17/1000, 2)))
print('C18  H3PO4  = {0}  ml / L'.format(round(out[17]*MM18/1000, 2)))
print('C19  NaCl   = {0}   g / L'.format(round(out[18]*MM19/1000, 2)))

print()
print('C1   KNO3   = {0}  mM'.format(round(out[0] , 2)))
print('C2   K2SO4  = {0}  mM'.format(round(out[1] , 2)))
print('C3   KH2PO4 = {0}  mM'.format(round(out[2] , 2)))
print('C4   MgNO3  = {0}  mM'.format(round(out[3] , 2)))
print('C5   MgSO4  = {0}  mM'.format(round(out[4] , 2)))
print('C6   CaNO3  = {0}  mM'.format(round(out[5] , 2)))
print('C7   HNO3   = {0}  mM'.format(round(out[6] , 2)))
print('C8   H2SO4  = {0}  mM'.format(round(out[7] , 2)))
print('C9   CaCl2  = {0}  mM'.format(round(out[8] , 2)))
print('C10  NaNO3  = {0}  mM'.format(round(out[9] , 2)))
print('C11  KCl    = {0}  mM'.format(round(out[10], 2)))
print('C12  HCl    = {0}  mM'.format(round(out[11], 2))) 
print('C13  KOH    = {0}  mM'.format(round(out[12], 2)))
print('C14  CaOH   = {0}  mM'.format(round(out[13], 2)))
print('C15  NaOH   = {0}  mM'.format(round(out[14], 2)))
print('C16  K2HPO4 = {0}  mM'.format(round(out[15], 2)))
print('C17  K3PO4  = {0}  mM'.format(round(out[16], 2)))
print('C18  H3PO4  = {0}  mM'.format(round(out[17], 2)))
print('C19  NaCl   = {0}  mM'.format(round(out[18], 2)))

C1, C2, C3, C4, C5, C6, C7, C8, C9, C10, C11, C12, C13, C14, C15, C16, C17, C18, C19 = out
eq1 = abs(C1 + 2*C6 + 2*C4 + C10 + C7)                                                     #  N balance
eq2 = abs(C1 + C3 + 2*C2 + C11 + C13 + 2*C16 + 3*C17)                                      #  K balance
eq3 = abs(C6 + C9 + C14)                                                                   #  Ca balance
eq4 = abs(C3 + C16 + C17 + C18)                                                            #  P balance
eq5 = abs(C5 + C2 + C8)                                                                    #  S balance
eq6 = abs(C5 + C4)                                                                         #  Mg balance
eq7 = abs(C7 + C12 + 2*C8 - C13 - 2*C14 - C15 - C16 - 2*C17 + 2*C18)                       #  H balance
eq8 = abs(2*C9 + C11 + C12 + C19)                                                          #  Cl balance
eq9 = abs(C10 + C15 + C19)                                                                 #  Na balance

print()
print('N     {0}'.format(round(eq1, 3)))
print('K     {0}'.format(round(eq2, 3)))
print('Ca    {0}'.format(round(eq3, 3)))
print('P     {0}'.format(round(eq4, 3)))
print('S     {0}'.format(round(eq5, 3)))
print('Mg    {0}'.format(round(eq6, 3)))
print('H     {0}'.format(round(eq7, 3)))
print('Cl    {0}'.format(round(eq8, 3)))
print('Na    {0}'.format(round(eq9, 3)))

convergence:  True
objective function value =  1.6e-05

The dosing solution should contain:

C1   KNO3   = 2.51   g / L
C2   K2SO4  = 0.17   g / L
C3   KH2PO4 = 0.79   g / L
C4   MgNO3  = 1.24   g / L
C5   MgSO4  = 0.04   g / L
C6   CaNO3  = 4.33   g / L
C7   HNO3   = 1.82  ml / L
C8   H2SO4  = 0.48  ml / L
C9   CaCl2  = 0.24   g / L
C10  NaNO3  = 0.0   g / L
C11  KCl    = 0.11   g / L
C12  HCl    = 0.43  ml / L
C13  KOH    = 0.0   g / L
C14  CaOH   = 0.0   g / L
C15  NaOH   = 0.0   g / L
C16  K2HPO4 = 0.29   g / L
C17  K3PO4  = 0.0   g / L
C18  H3PO4  = 0.0  ml / L
C19  NaCl   = 0.0   g / L

C1   KNO3   = 24.87  mM
C2   K2SO4  = 0.98  mM
C3   KH2PO4 = 5.84  mM
C4   MgNO3  = 4.82  mM
C5   MgSO4  = 0.18  mM
C6   CaNO3  = 18.33  mM
C7   HNO3   = 28.81  mM
C8   H2SO4  = 8.85  mM
C9   CaCl2  = 1.65  mM
C10  NaNO3  = 0.0  mM
C11  KCl    = 1.48  mM
C12  HCl    = 5.22  mM
C13  KOH    = 0.0  mM
C14  CaOH   = 0.02  mM
C15  NaOH   = 0.0  mM
C16  K2HPO4 = 1.65  mM
C17  K3PO4  = 0.02  mM
C18  H3PO